In [13]:
import os, json, random
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import torchaudio.transforms as T
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import models
from torch.optim.lr_scheduler import StepLR
import numpy as np
from datetime import datetime
from transformers import (
    Wav2Vec2FeatureExtractor,
    HubertModel,
)
from textgrids import TextGrid
from tqdm import tqdm

In [14]:
pinyin_list = ['a', 'ai', 'an', 'ang', 'ao', 'ba', 'bai', 'ban', 'bang', 'bao', 'be', 'bei', 'ben', 'beng', 'bi', 'bian', 'bianr', 'biao', 'bie', 'bin', 'bing', 'bo', 'bu', 'ca', 'cai', 'can', 'cang', 'cao', 'ce', 'cen', 'ceng', 'cha', 'chai', 'chan', 'chang', 'chao', 'che', 'chen', 'cheng', 'chi', 'chong', 'chou', 'chu', 'chua', 'chuai', 'chuan', 'chuang', 'chui', 'chun', 'chuo', 'ci', 'cong', 'cou', 'cu', 'cuan', 'cui', 'cun', 'cuo', 'da', 'dai', 'dan', 'dang', 'dao', 'de', 'dei', 'den', 'deng', 'di', 'dia', 'dian', 'diao', 'die', 'ding', 'diu', 'dong', 'dou', 'du', 'duan', 'dui', 'dun', 'duo', 'e', 'ei', 'en', 'ng', 'eng', 'er', 'fa', 'fan', 'fang', 'fei', 'fen', 'feng', 'fo', 'fou', 'fu', 'ga', 'gai', 'gan', 'gang', 'gao', 'ge', 'gei', 'gen', 'geng', 'gong', 'gou', 'gu', 'gua', 'guai', 'guan', 'guang', 'gui', 'gun', 'guo', 'ha', 'hai', 'hair', 'han', 'hang', 'hao', 'he', 'hei', 'hen', 'heng', 'hong', 'hou', 'hu', 'hua', 'huai', 'huan', 'huang', 'hui', 'huir', 'hun', 'huo', 'ji', 'jia', 'jian', 'jiang', 'jiao', 'jie', 'jin', 'jing', 'jiong', 'jiu', 'ju', 'juan', 'jue', 'jun', 'ka', 'kai', 'kan', 'kang', 'kao', 'ke', 'kei', 'ken', 'keng', 'kong', 'kou', 'ku', 'kua', 'kuai', 'kuan', 'kuang', 'kui', 'kun', 'kuo', 'la', 'lai', 'lan', 'lang', 'lao', 'le', 'lei', 'leng', 'li', 'lia', 'lian', 'liang', 'liao', 'lie', 'lin', 'ling', 'liu', 'lo', 'long', 'lou', 'lu', 'lv', 'luan', 'lve', 'lue', 'lun', 'luo', 'ma', 'mai', 'man', 'mang', 'mao', 'me', 'mei', 'men', 'meng', 'mi', 'mian', 'miao', 'mie', 'min', 'ming', 'miu', 'mo', 'mou', 'mu', 'na', 'nar', 'nai', 'nan', 'nang', 'nao', 'ne', 'nei', 'nen', 'neng', 'ni', 'nia', 'nian', 'niang', 'niao', 'nie', 'nin', 'ning', 'niu', 'nong', 'nou', 'nu', 'nv', 'nuan', 'nve', 'nue', 'nuo', 'o', 'ou', 'pa', 'pai', 'pan', 'pang', 'pao', 'pe', 'pei', 'pen', 'peng', 'pi', 'pian', 'piao', 'pie', 'pin', 'ping', 'po', 'pou', 'pu', 'qi', 'qia', 'qian', 'qiang', 'qiao', 'qie', 'qin', 'qing', 'qiong', 'qiu', 'qu', 'quan', 'que', 'qun', 'ran', 'rang', 'rao', 're', 'ren', 'reng', 'ri', 'rong', 'rou', 'ru', 'rua', 'ruan', 'rui', 'run', 'ruo', 'sa', 'sai', 'san', 'sang', 'sao', 'se', 'sen', 'seng', 'sha', 'shai', 'shan', 'shang', 'shao', 'she', 'shei', 'shen', 'sheng', 'shi', 'shir', 'shou', 'shu', 'shua', 'shuai', 'shuan', 'shuang', 'shui', 'shun', 'shuo', 'si', 'song', 'sou', 'su', 'suan', 'sui', 'sun', 'suo', 'ta', 'tai', 'tan', 'tang', 'tao', 'te', 'tei', 'teng', 'ti', 'tian', 'tianr', 'tiao', 'tie', 'ting', 'tong', 'tou', 'tu', 'tuan', 'tui', 'tun', 'tuo', 'wa', 'wai', 'wan', 'wanr', 'wang', 'wei', 'weir', 'wen', 'weng', 'wo', 'wu', 'xi', 'xia', 'xian', 'xiang', 'xiao', 'xie', 'xin', 'xing', 'xiong', 'xiu', 'xu', 'xuan', 'xue', 'xun', 'ya', 'yan', 'yang', 'yao', 'ye', 'yi', 'yin', 'ying', 'yo', 'yong', 'you', 'yu', 'yuan', 'yue', 'yun', 'za', 'zai', 'zan', 'zang', 'zao', 'ze', 'zei', 'zen', 'zeng', 'zha', 'zhai', 'zhan', 'zhang', 'zhao', 'zhe', 'zhei', 'zhen', 'zheng', 'zher', 'zhi', 'zhong', 'zhou', 'zhu', 'zhua', 'zhuai', 'zhuan', 'zhuang', 'zhui', 'zhun', 'zhuo', 'zi', 'zong', 'zou', 'zu', 'zuan', 'zui', 'zun', 'zuo', 'gair', 'yir', 'jinr', 'menr', 'her', 'gunr', 'huanr', 'jiar', 'duir', 'dir', 'banr', 'yanr', 'cir', 'genr', 'ger', 'tongr', 'renr', 'kuair', 'yuanr', 'liar', 'dunr', 'fur', 'zaor', 'yangr', 'ter', 'yingr', 'fanr', 'tuor', 'pir', 'jingr', 'hanr', 'duor', 'tanr', 'zhunr', 'jir', 'huor', 'far', 'dianr', 'zhur', 'tour', 'qir', 'mingr', 'lanr', 'lir', 'daor', 'niur']

pinyin_to_label = {p: i for i, p in enumerate(pinyin_list)}
label_to_pinyin = {i: p for i, p in enumerate(pinyin_list)}

In [15]:

def change_layers(model):
    model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    model.fc = nn.Linear(512, len(pinyin_list))
    return model

model = models.resnet18(pretrained=False)
change_layers(model)


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [16]:
# model.load_state_dict(torch.load('../checkpoints/07_09_2023_14:46:23/ckp_18.pt')['model_state_dict'])
model.load_state_dict(torch.load('../checkpoints/13_09_2023_22:13:20/ckp_12.pt')['model_state_dict'])

<All keys matched successfully>

In [17]:
model.eval()
model = model.cuda()

In [18]:
hubert_model_path = "TencentGameMate/chinese-hubert-base"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(hubert_model_path)
hubert_model = HubertModel.from_pretrained(hubert_model_path)

hubert_model = hubert_model.cuda()
hubert_model.eval()

HubertModel(
  (feature_extractor): HubertFeatureEncoder(
    (conv_layers): ModuleList(
      (0): HubertGroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1): HubertNoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (2): HubertNoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (3): HubertNoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (4): HubertNoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5): HubertNoLayerNormConvLayer(

In [19]:
metadata = json.load(open('test_py_list.json'))

metadata[0]

{'file_name': '008002',
 'line_text': '喜欢把李敖的书插在牛仔裤的前面？',
 'gt_py': 'xi3 huan1 ba2 li3 ao2 de5 shu1 cha1 zai4 niu2 zai3 ku4 de5 qian2 mian4',
 'line_py': ['xi3',
  'huan5',
  'ba3',
  'li3',
  'ao2',
  'de5',
  'shu1',
  'cha1',
  'zai4',
  'niu2',
  'zai3',
  'ku4',
  'de5',
  'qian2',
  'mian4'],
 'hetero_py': [['xi3'],
  ['huan1'],
  ['ba3', 'ba4'],
  ['li3'],
  ['ao2'],
  ['de5', 'di2', 'di4'],
  ['shu1'],
  ['cha1'],
  ['zai4'],
  ['niu2'],
  ['zi1', 'zi3', 'zai3'],
  ['ku4'],
  ['de5', 'di2', 'di4'],
  ['qian2'],
  ['mian4']],
 'g2pm_py': ['xi3',
  'huan1',
  'ba3',
  'li3',
  'ao2',
  'de5',
  'shu1',
  'cha1',
  'zai4',
  'niu2',
  'zai3',
  'ku4',
  'de5',
  'qian2',
  'mian4']}

In [20]:
file_keys = [m['file_name'] for m in metadata]

file_keys[:2]

['008002', '008003']

In [21]:

grid_dir = './biaobei_aligned'
grids = os.listdir(grid_dir)

py_list = []

for g in grids:
    file_name = g.split('.')[0]
    if file_name not in file_keys:
        continue
    grid = TextGrid(os.path.join(grid_dir, g))    
    pds = []
    for w in grid['words']:
        # if len(w.text) > 0 and w.xmax - w.xmin > 0.7:
        #     print(g, w.text, w.xmin, w.xmax)            
        if len(w.text) > 0 and w.xmax - w.xmin > 0.03 and w.xmax - w.xmin < 0.6:
            pds.append((w.text, w.xmin, w.xmax))
        elif w.text != '':
            print(g, w)
    py_list.append({
        'file_name': file_name,
        'pinyin_clips': pds
    })
    

008493.TextGrid <Interval text="shuai4" xmin=1.77 xmax=1.79>
008233.TextGrid <Interval text="hang2" xmin=1.16 xmax=1.18>
008945.TextGrid <Interval text="lo5" xmin=4.72 xmax=4.74>
008926.TextGrid <Interval text="hai2" xmin=4.67 xmax=4.69>
009987.TextGrid <Interval text="mei3" xmin=0.83 xmax=0.85>
009344.TextGrid <Interval text="suo1" xmin=1.0 xmax=1.02>
008557.TextGrid <Interval text="he2" xmin=3.44 xmax=3.46>
008914.TextGrid <Interval text="pu3" xmin=0.73 xmax=0.75>
009708.TextGrid <Interval text="di4" xmin=3.14 xmax=3.16>
009757.TextGrid <Interval text="leng2" xmin=2.36 xmax=2.38>
009860.TextGrid <Interval text="a1" xmin=1.47 xmax=1.48>
009860.TextGrid <Interval text="a1" xmin=5.01 xmax=5.02>
009199.TextGrid <Interval text="ao3" xmin=3.32 xmax=3.33>
009649.TextGrid <Interval text="mu3" xmin=1.01 xmax=1.03>
009841.TextGrid <Interval text="di4" xmin=1.99 xmax=2.01>


In [22]:
audio_py_pairs = {}
for p in py_list:
    file_path = 'lab_wav_pairs/' +  p['file_name'] + '.wav'
    for pc in p['pinyin_clips']:
        if file_path in audio_py_pairs:
            audio_py_pairs[file_path].append((pc[0], pc[1], pc[2]))
        else:
            audio_py_pairs[file_path] = [(pc[0], pc[1], pc[2])]

In [23]:
audio_py_hetero_list = {}
for m in metadata:
    filename = 'lab_wav_pairs/' + m['file_name'] + '.wav'
    m_hetero_py = []
    for hp in m['hetero_py']:
        pys = [p[:-1] for p in hp]
        m_hetero_py.append(pys)
    audio_py_hetero_list[filename] = {
        'nm_hetero_py': m_hetero_py,
        'm_hetero_py': m['hetero_py']
    }
audio_py_hetero_list.keys()

dict_keys(['lab_wav_pairs/008002.wav', 'lab_wav_pairs/008003.wav', 'lab_wav_pairs/008004.wav', 'lab_wav_pairs/008005.wav', 'lab_wav_pairs/008006.wav', 'lab_wav_pairs/008007.wav', 'lab_wav_pairs/008008.wav', 'lab_wav_pairs/008009.wav', 'lab_wav_pairs/008010.wav', 'lab_wav_pairs/008011.wav', 'lab_wav_pairs/008012.wav', 'lab_wav_pairs/008013.wav', 'lab_wav_pairs/008014.wav', 'lab_wav_pairs/008015.wav', 'lab_wav_pairs/008016.wav', 'lab_wav_pairs/008017.wav', 'lab_wav_pairs/008018.wav', 'lab_wav_pairs/008019.wav', 'lab_wav_pairs/008020.wav', 'lab_wav_pairs/008021.wav', 'lab_wav_pairs/008022.wav', 'lab_wav_pairs/008023.wav', 'lab_wav_pairs/008024.wav', 'lab_wav_pairs/008025.wav', 'lab_wav_pairs/008027.wav', 'lab_wav_pairs/008028.wav', 'lab_wav_pairs/008029.wav', 'lab_wav_pairs/008030.wav', 'lab_wav_pairs/008031.wav', 'lab_wav_pairs/008032.wav', 'lab_wav_pairs/008033.wav', 'lab_wav_pairs/008034.wav', 'lab_wav_pairs/008035.wav', 'lab_wav_pairs/008036.wav', 'lab_wav_pairs/008037.wav', 'lab_wav_

In [24]:
audio_py_hetero_list['lab_wav_pairs/008002.wav']

{'nm_hetero_py': [['xi'],
  ['huan'],
  ['ba', 'ba'],
  ['li'],
  ['ao'],
  ['de', 'di', 'di'],
  ['shu'],
  ['cha'],
  ['zai'],
  ['niu'],
  ['zi', 'zi', 'zai'],
  ['ku'],
  ['de', 'di', 'di'],
  ['qian'],
  ['mian']],
 'm_hetero_py': [['xi3'],
  ['huan1'],
  ['ba3', 'ba4'],
  ['li3'],
  ['ao2'],
  ['de5', 'di2', 'di4'],
  ['shu1'],
  ['cha1'],
  ['zai4'],
  ['niu2'],
  ['zi1', 'zi3', 'zai3'],
  ['ku4'],
  ['de5', 'di2', 'di4'],
  ['qian2'],
  ['mian4']]}

# TOP-1 Only

In [ ]:

correct = 0
total = 0
err_list = []

for file_path in tqdm(list(audio_py_pairs.keys())):
    for index, ap in enumerate(audio_py_pairs[file_path]):
        py, start, end = ap
        try:
            SPEECH_WAVEFORM, SAMPLE_RATE = torchaudio.load(file_path, int(start * 48000), int((end-start) * 48000))
            SPEECH_WAVEFORM = torchaudio.functional.resample(SPEECH_WAVEFORM, SAMPLE_RATE, 16000)

            SPEECH_WAVEFORM = F.pad(SPEECH_WAVEFORM, (0, 16000-SPEECH_WAVEFORM.shape[1]), mode='constant', value=0)

            with torch.no_grad():
                # features, _ = hubert_model.extract_features(SPEECH_WAVEFORM.cuda())
                input_values = feature_extractor(SPEECH_WAVEFORM[0], return_tensors="pt", sampling_rate=16000).input_values
                outputs = hubert_model(input_values.cuda())
                last_hidden_state = outputs.last_hidden_state
            
            # print(last_hidden_state.shape)
            with torch.no_grad():
                output = model(torch.unsqueeze(last_hidden_state, 0))
            _, predicted = torch.topk(output, 1)
            # print(predicted, label_to_pinyin[predicted[0][0].item()], py)
            c_sum = sum([1 if label_to_pinyin[predicted[0][i].item()] == py[:-1] else 0 for i in range(len(predicted))])
            correct += c_sum
            if c_sum == 0:
                for i in range(len(predicted)):
                    if py[:-1] != label_to_pinyin[predicted[0][i].item()]:
                        print(file_path, index, start, end, label_to_pinyin[predicted[0][i].item()], py[:-1])
                        err_list.append((file_path, index, start, end, label_to_pinyin[predicted[0][i].item()], py[:-1]))
            total += 1
        except Exception as e:
            print(file_path, ap, e)
            continue

print('Accuracy of the model: %.3f %%' %((100*correct)/(total)))


In [14]:

detected_err_list = []

for err in err_list:
    filename, index, start, end, predicted, gt = err
    print(predicted, gt)
    print(filename, audio_py_hetero_list[filename]['nm_hetero_py'][index], audio_py_hetero_list[filename]['m_hetero_py'][index])
    nm_hetero_py_i = audio_py_hetero_list[filename]['nm_hetero_py'][index]
    m_hetero_py_i = audio_py_hetero_list[filename]['m_hetero_py'][index]
    if predicted in nm_hetero_py_i:
        hetero_index = nm_hetero_py_i.index(predicted)
        print('!!!')
        audio_py_pairs[filename][index] = (m_hetero_py_i[hetero_index], audio_py_pairs[filename][index][1], audio_py_pairs[filename][index][2])
        detected_err_list.append(err)
    

yuan yue
lab_wav_pairs/009419.wav ['yue'] ['yue4']
da dai
lab_wav_pairs/008493.wav ['hui', 'kuai'] ['hui4', 'kuai4']
jiang jia
lab_wav_pairs/008763.wav ['jia', 'jia'] ['jia3', 'jia4']
shang sheng
lab_wav_pairs/009497.wav ['sheng'] ['sheng1']
lie lian
lab_wav_pairs/009668.wav ['lian'] ['lian2']
shun chi
lab_wav_pairs/008822.wav ['chi'] ['chi2']
chan chai
lab_wav_pairs/008336.wav ['chai', 'ca'] ['chai1', 'ca1']
hu huo
lab_wav_pairs/008336.wav ['huo'] ['huo4']
zhuan zhuai
lab_wav_pairs/009274.wav ['zhuai', 'zhuai', 'ye'] ['zhuai4', 'zhuai1', 'ye4']
min ming
lab_wav_pairs/008895.wav ['ming'] ['ming2']
ban dan
lab_wav_pairs/008895.wav ['dan'] ['dan3']
ming min
lab_wav_pairs/009302.wav ['min'] ['min3']
jian jie
lab_wav_pairs/009510.wav ['jie', 'jie', 'xie'] ['jie3', 'jie4', 'xie4']
hu hun
lab_wav_pairs/009856.wav ['hun'] ['hun2']
dou bao
lab_wav_pairs/008547.wav ['bao'] ['bao3']
suo song
lab_wav_pairs/008547.wav ['song'] ['song4']
xing xin
lab_wav_pairs/008661.wav ['xin', 'shen'] ['xin4', 's

In [15]:
g2pa_pairs = {}

for ap in audio_py_pairs:
    file_name = ap.split('/').pop().split('.')[0]
    py_list = ' '.join([a[0] for a in audio_py_pairs[ap]])
    print(file_name, py_list)
    g2pa_pairs[file_name] = py_list
    

009419 guai4 bu4 de2 gu3 ren2 chang2 shuo1 jin4 zhu1 zhe3 chi4 wo3 xian4 zai4 que4 shi2 yue4 lai2 yue4 xiang4 zhu3 ren2 le5
008493 shi4 shi2 shang4 gao1 li2 hun1 yi3 jing1 gei3 ke4 luo2 di4 ya4 de5 she4 hui4 dai4 lai2 le5 hen3 da4 de5 ying2 xiang3
009112 yuan2 lai2 ta1 duo3 qi3 lai2 wan2 wei1 bo2 yu3 you3 ren2 da4 tan2 xing1 zuo4
009334 bei4 cheng1 zan4 le5 zhen1 kai1 xin1 ya1
008763 hei1 zuo4 fang1 nei4 zheng4 zai4 fen1 zhuang1 jia3 mao4 shui3 ni2
009497 shu4 ren2 xiong1 ke3 xi1 wan3 sheng1 le5 yi4 bai3 nian2 huo4 zhe3 zao3 sheng1 le5 yi4 bai3 nian2
009668 wu2 yi2 tang2 jun4 de5 shuai2 shou3 zou3 ren2 dui4 lian2 you2 wang3 luo4 lai2 shuo1 shi4 xue3 shang4 jia1 shuang1
009872 cao2 hong2 liang4 shuo1 hai2 you3 ji3 ge4 kong1 zhe5 mei2 guan1 xi4 rang4 wo3 chao1 chao1
008073 ge4 ti3 gong1 shang1 hu4 gen1 ju4 jing1 ying2 xu1 yao4 ke2 yi3 pin4 qing3 ruo4 gan1 bang1 shou3 huo4 zhe3 xue2 tu2
008822 shuang1 fang1 shang1 ding4 ji4 xu4 jiu4 ci3 wen4 ti2 bao3 chi2 cuo1 shang1
008846 ping2 shi2 wei

In [16]:

json_object = json.dumps(g2pa_pairs, indent=4)

with open("g2pa_pairs.json", "w") as outfile:
    outfile.wrsite(json_object)

# Compare Top-K

In [26]:

correct = 0
total = 0
err_list = []

for file_path in tqdm(list(audio_py_pairs.keys())):
    for index, ap in enumerate(audio_py_pairs[file_path]):
        py, start, end = ap
        try:
            SPEECH_WAVEFORM, SAMPLE_RATE = torchaudio.load(file_path, int(start * 48000), int((end-start) * 48000))
            SPEECH_WAVEFORM = torchaudio.functional.resample(SPEECH_WAVEFORM, SAMPLE_RATE, 16000)

            SPEECH_WAVEFORM = F.pad(SPEECH_WAVEFORM, (0, 16000-SPEECH_WAVEFORM.shape[1]), mode='constant', value=0)

            with torch.no_grad():
                # features, _ = hubert_model.extract_features(SPEECH_WAVEFORM.cuda())
                input_values = feature_extractor(SPEECH_WAVEFORM[0], return_tensors="pt", sampling_rate=16000).input_values
                outputs = hubert_model(input_values.cuda())
                last_hidden_state = outputs.last_hidden_state
            
            # print(last_hidden_state.shape)
            with torch.no_grad():
                output = model(torch.unsqueeze(last_hidden_state, 0))
            _, predicted = torch.topk(output, 5)
            # print(predicted, label_to_pinyin[predicted[0][0].item()], py)
            # c_sum = sum([1 if label_to_pinyin[predicted[0][i].item()] == py[:-1] else 0 for i in range(len(predicted))])
            c_sum = 1 if label_to_pinyin[predicted[0][0].item()] == py[:-1] else 0
            correct += c_sum
            if c_sum == 0:
                # for i in range(len(predicted)):
                #     if py[:-1] != label_to_pinyin[predicted[0][i].item()]:
                print(file_path, index, start, end, label_to_pinyin[predicted[0][0].item()], py[:-1])
                err_list.append((file_path, index, start, end, predicted[0], py[:-1]))
            total += 1
        except Exception as e:
            print(file_path, ap, e)
            continue


  0%|          | 1/1944 [00:00<14:50,  2.18it/s]

lab_wav_pairs/009419.wav 16 4.44 4.56 yuan yue


  0%|          | 2/1944 [00:00<13:01,  2.49it/s]

lab_wav_pairs/008493.wav 16 3.76 3.89 da dai


  0%|          | 5/1944 [00:01<07:54,  4.09it/s]

...

 99%|█████████▉| 1933/1944 [07:40<00:02,  3.71it/s]

lab_wav_pairs/009135.wav 12 3.59 3.76 jian jin


100%|█████████▉| 1942/1944 [07:42<00:00,  3.55it/s]

lab_wav_pairs/009240.wav 5 1.48 1.65 ga jia
lab_wav_pairs/009059.wav 9 2.42 2.58 xia xiang


100%|██████████| 1944/1944 [07:43<00:00,  4.20it/s]


In [27]:
print('Accuracy of the model: %.3f %%' %((100*correct)/(total)))

Accuracy of the model: 96.545 %


In [50]:

detected_err_list = []

for err in err_list:
    filename, index, start, end, predicted, gt = err
    # print(predicted, gt)
    # for p in predicted:
    #     print(label_to_pinyin[p.item()])
    pp = label_to_pinyin[predicted[0].item()]
    # print(filename, audio_py_hetero_list[filename]['nm_hetero_py'][index], audio_py_hetero_list[filename]['m_hetero_py'][index])
    nm_hetero_py_i = audio_py_hetero_list[filename]['nm_hetero_py'][index]
    m_hetero_py_i = audio_py_hetero_list[filename]['m_hetero_py'][index]
    if pp in nm_hetero_py_i:
        hetero_index = nm_hetero_py_i.index(pp)
        audio_py_pairs[filename][index] = (m_hetero_py_i[hetero_index], audio_py_pairs[filename][index][1], audio_py_pairs[filename][index][2])
        detected_err_list.append(err)
    else:
        pp = label_to_pinyin[predicted[1].item()]
        if pp in nm_hetero_py_i:
            hetero_index = nm_hetero_py_i.index(pp)
            audio_py_pairs[filename][index] = (m_hetero_py_i[hetero_index], audio_py_pairs[filename][index][1], audio_py_pairs[filename][index][2])
            detected_err_list.append(err)
        else:
            pp = label_to_pinyin[predicted[2].item()]
            if pp in nm_hetero_py_i:
                hetero_index = nm_hetero_py_i.index(pp)
                audio_py_pairs[filename][index] = (m_hetero_py_i[hetero_index], audio_py_pairs[filename][index][1], audio_py_pairs[filename][index][2])
                detected_err_list.append(err)
            else:
                pp = label_to_pinyin[predicted[3].item()]
                if pp in nm_hetero_py_i:
                    hetero_index = nm_hetero_py_i.index(pp)
                    audio_py_pairs[filename][index] = (m_hetero_py_i[hetero_index], audio_py_pairs[filename][index][1], audio_py_pairs[filename][index][2])
                    detected_err_list.append(err)
                else:
                    pp = label_to_pinyin[predicted[4].item()]
                    if pp in nm_hetero_py_i:
                        hetero_index = nm_hetero_py_i.index(pp)
                        audio_py_pairs[filename][index] = (m_hetero_py_i[hetero_index], audio_py_pairs[filename][index][1], audio_py_pairs[filename][index][2])
                        detected_err_list.append(err)
    

In [51]:
g2pa_pairs = {}

for ap in audio_py_pairs:
    file_name = ap.split('/').pop().split('.')[0]
    py_list = ' '.join([a[0] for a in audio_py_pairs[ap]])
    print(file_name, py_list)
    g2pa_pairs[file_name] = py_list
    

009419 guai4 bu4 de2 gu3 ren2 chang2 shuo1 jin4 zhu1 zhe3 chi4 wo3 xian4 zai4 que4 shi2 yue4 lai2 yue4 xiang4 zhu3 ren2 le5
008493 shi4 shi2 shang4 gao1 li2 hun1 yi3 jing1 gei3 ke4 luo2 di4 ya4 de5 she4 hui4 dai4 lai2 le5 hen3 da4 de5 ying2 xiang3
009112 yuan2 lai2 ta1 duo3 qi3 lai2 wan2 wei1 bo2 yu3 you3 ren2 da4 tan2 xing1 zuo4
009334 bei4 cheng1 zan4 le5 zhen1 kai1 xin1 ya1
008763 hei1 zuo4 fang1 nei4 zheng4 zai4 fen1 zhuang1 jia3 mao4 shui3 ni2
009497 shu4 ren2 xiong1 ke3 xi1 wan3 sheng1 le5 yi4 bai3 nian2 huo4 zhe3 zao3 sheng1 le5 yi4 bai3 nian2
009668 wu2 yi2 tang2 jun4 de5 shuai2 shou3 zou3 ren2 dui4 lian2 you2 wang3 luo4 lai2 shuo1 shi4 xue3 shang4 jia1 shuang1
009872 cao2 hong2 liang4 shuo1 hai2 you3 ji3 ge4 kong1 zhe5 mei2 guan1 xi4 rang4 wo3 chao1 chao1
008073 ge4 ti3 gong1 shang1 hu4 gen1 ju4 jing1 ying2 xu1 yao4 ke2 yi3 pin4 qing3 ruo4 gan1 bang1 shou3 huo4 zhe3 xue2 tu2
008822 shuang1 fang1 shang1 ding4 ji4 xu4 jiu4 ci3 wen4 ti2 bao3 chi2 cuo1 shang1
008846 ping2 shi2 wei

In [52]:

json_object = json.dumps(g2pa_pairs, indent=4)

with open("g2pa_pairs4.json", "w") as outfile:
    outfile.write(json_object)